![Banner](images/banner.png)

# Queries - SELECT and WITH Statements

In [ ]:
import oracledb

In [ ]:
un       = "pythondemo"
pw       = "welcome"

adminun  = "system"
adminpw  = "oracle"

cs       = "localhost/orclpdb1"

connection = oracledb.connect(user=un, password=pw, dsn=cs)

systemconnection = oracledb.connect(user=adminun, password=adminpw, dsn=cs)

# Fetching Rows

Documentation reference link: [Fetch Methods](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#fetch-methods)

In [ ]:
# Fetching single rows

with connection.cursor() as cursor:
    
    cursor.execute("select * from SampleQueryTab where id = :1", [6])
    
    row = cursor.fetchone()
    print(row)
    

In [ ]:
# Fetching all rows from a small table of not too big size

with connection.cursor() as cursor:
       
    cursor.execute("select * from SampleQueryTab")
    
    rows = cursor.fetchall()
    
    for r in rows:
        print(r)
      

In [ ]:
# Fetching all rows from a big table

with connection.cursor() as cursor:
 
    cursor.execute("select * from SampleQueryTab")  # pretend this is big
    
    while True:
        rows = cursor.fetchmany()  # get a batch rows (of size cursor.arraysize=100)
        for r in rows:
            print(r)
        if len(rows) < cursor.arraysize:
            break


## A key tuning goal is to reduce "round-trips"


> "A server round-trip is defined as the trip from the client to the server and back to the client."





![Round-trip](images/roundtrip.png)

Round-trips affect performance and system scalability.

Make every round-trip useful:

- fetch multiple rows at a time
- insert multiple rows at a time
- don't select more data than needed
- don't overuse commit or rollback

## Round-trip comparison - prefetchrows & arraysize

Documentation reference link: [Tuning python-oracledb](https://python-oracledb.readthedocs.io/en/latest/user_guide/tuning.html)

Regardless of which fetch method is used there are two tuning parameters that affect internal buffering of fetched rows. This alters the number of round-trips required to fetch all the query data from the database.  These parameters do not affect how, or when, rows are returned to the application itself.

Here's a demo counting round-trips:

In [ ]:
def get_session_id(connection):
    sid, = connection.cursor().execute("SELECT sys_context('userenv', 'sid') FROM dual").fetchone()
    return sid

def get_round_trips(systemconnection, sid):
    sql = """SELECT ss.value
             FROM v$sesstat ss, v$statname sn
             WHERE ss.sid = :sid
             AND ss.statistic# = sn.statistic#
             AND sn.name LIKE '%roundtrip%client%'"""  
    roundtrips, = systemconnection.cursor().execute(sql, [sid]).fetchone()
    return roundtrips

sid = get_session_id(connection)

def do_query(connection, numrows, prefetchrows, arraysize):

    roundtrips = get_round_trips(systemconnection, sid)

    with connection.cursor() as cursor:
        cursor.prefetchrows = prefetchrows
        cursor.arraysize    = arraysize
        cursor.execute("select * from all_objects where rownum <= :nr", [numrows])
        rows = cursor.fetchall()

    roundtrips = get_round_trips(systemconnection, sid) - roundtrips

    print("Number of rows: {:5}, prefetchrows {:4}  arraysize {:4}  roundtrips {:3} ".format(len(rows), prefetchrows, arraysize, roundtrips))


# do_query(connection, number of rows, prefetchrows, arraysize)

print("Default prefetch & arraysize values:")
do_query(connection,                1,            2,       100)
do_query(connection,              100,            2,       100)
do_query(connection,             1000,            2,       100)
do_query(connection,            10000,            2,       100)

print("\n'Unknown' large number of rows:")
do_query(connection,            10000,            2,      1000)
do_query(connection,            10000,         1000,      1000)

print("\n'Page' of rows:")
do_query(connection,               20,           20,        20)
do_query(connection,               20,           21,        20)



Your tuning goal is to reduce round-trips (thus improving performance) without using too much internal buffer memory.

When selecting a huge number of rows, tuning `arraysize` is important. In general there is no need to set `prefetchrows` in this scenario.

When selecting a known, small number of rows such as for 'paging' through a result set, then set `prefetchrows` to be one larger than the number of rows returned.

# Querying LOB Columns

Documentation reference link: [Using CLOB and BLOB Data](https://python-oracledb.readthedocs.io/en/latest/user_guide/lob_data.html)

When fetching LOBs there are two modes that can be used:
- Fetching as a "locator" for streaming use
- Fetching directly as a string or buffer

### Fetching LOBs for streaming

- this is the default
- it requires more round trips and has more overhead
- it is useful when very long LOBs are being used

In [ ]:
with connection.cursor() as cursor:

    id = 1
    textData = "The quick brown fox jumps over the lazy dog"
    cursor.execute("truncate table TestClobs")
    cursor.execute("insert into TestClobs (IntCol, ClobCol) values (:1, :2)", [id, textData])

    cursor.execute("select ClobCol from TestClobs where IntCol = :ic1", [id])
    c, = cursor.fetchone()
    print("CLOB length:", c.size())
    print("CLOB data:", c.read())

### Fetching CLOBs as String

- this is much faster than the streaming method
- LOBs are limited to 1 GB (and obviously must all be in memory at once)

In [ ]:
oracledb.defaults.fetch_lobs = False

with connection.cursor() as cursor:
    id = 1
    cursor.execute("select ClobCol from TestClobs where IntCol = :ic2", [id])
    c, = cursor.fetchone()

    print("CLOB length:", len(c))
    print("CLOB data:", c)

The same `defaults` setting will return BLOBs as buffers.

### Sample benchmark
A sample benchmark shows the performance benefit of querying using the direct String method compared with using the default locator stream method. 
![LOB Benchmark](images/lob-benchmark.png)

# Fetching Numbers as Decimals

Oracle's NUMBER format is Decimal but Python uses floating point so there can be conversion artifacts.

Documentation reference link:  [Fetched Number Precision](https://python-oracledb.readthedocs.io/en/latest/user_guide/sql_execution.html#fetched-number-precision)

In [ ]:
with connection.cursor() as cursor:
    cursor.execute("select 0.1 as d1 from dual")
    v, = cursor.fetchone()
    print('Value =', v, '\tValue * 3 =', v * 3)

Fetching as Decimal may be useful.

In [ ]:
import decimal

oracledb.defaults.fetch_decimals = True

with connection.cursor() as cursor:
    cursor.execute("select 0.1 as d2 from dual")
    v, = cursor.fetchone()
    print('Value =', v, '\tValue * 3 =', v * 3)

## Multiple Values in "WHERE ... IN" Clauses

Documentation reference link: [Binding Multiple Values to a SQL WHERE IN Clause](https://python-oracledb.readthedocs.io/en/latest/user_guide/bind.html#binding-multiple-values-to-a-sql-where-in-clause)

### A fixed number of binds

It's important to use bind variables for security.  For small numbers of binds this is simple.  One placeholder must be used for each value in the IN clause.

In [ ]:
sql = """select name
         from SampleQueryTab
         where id in (:id1, :id2, :id3, :id4, :id5)
         order by id"""

with connection.cursor() as cursor:
    cursor.execute(sql,
        id1=5, id2=7, id3=1, id4=3, id5=2)
    for row, in cursor:
        print(row)       

If statements are re-executed but the number of values varies, then pass None (i.e. NULL) for 'missing' values.  This lets the database execute the same statement text as before, which helps performance and scalability. 

In [ ]:
with connection.cursor() as cursor:
    cursor.execute(sql,
        id1=2, id2=4, id3=1, id4=None, id5=None)
    for row, in cursor:
        print(row) 

When using character data and the size of the values varies between statement execution, then using `setinputsizes()` with the column size or maximum expected input data size can help reduce the number of SQL 'versions' of the statement used by the database optimizer, although this is not often an issue and it's not uncommon to see SQL statements with tens or low hundreds of versions.

For example with the query:
```
sql = """select name
         from SampleQueryTab
         where name in (:n1, :n2, :n3, :n4, :n5)
         order by id"""
```
the statement
```
cursor.setinputsizes(n1=20, n2=20, n3=20, n4=20, n5=20)
```

indicates that each value bound will be a string of no more than 20 characters.

In [ ]:
sql = """select name
         from SampleQueryTab
         where name in (:n1, :n2, :n3, :n4, :n5)
         order by id"""

with connection.cursor() as cursor:
    cursor.setinputsizes(n1=20, n2=20, n3=20, n4=20, n5=20)
    cursor.execute(sql,
        n1='Anthony', n2='Barbie', n3='Bogus', n4=None, n5=None)
    for row, in cursor:
        print(row) 

### Dynamically creating the statement

If the number of bind values is not known, and the statement is never rexecuted, you can consider dynamically creating a statement.

In [ ]:
bind_values = [5, 7, 1, 3, 2, 6]
bind_names = [":" + str(i + 1) for i in range(len(bind_values))]
sql = """select name from SampleQueryTab where id in (%s)""" % (",".join(bind_names))
print(sql, "\n")
with connection.cursor() as cursor:
    cursor.execute(sql, bind_values)
    for row, in cursor:
        print(row)

### Binding using an Oracle type 

One solution when matching a huge number of values is to use the SQL `table()` clause and an Oracle type.  This works for up to 32K values.

In [ ]:
type_obj = connection.gettype("SYS.ODCINUMBERLIST")

with connection.cursor() as cursor:
    obj = type_obj.newobject()
    obj.extend([3, 4, 7])
    cursor.execute("""select name
                      from SampleQueryTab
                      where id in (select * from table(:1))""",
                [obj])
    for row, in cursor:
        print(row)

The presupplied types `SYS.ODCIVARCHAR2LIST` or `SYS.ODCIDATELIST` can similarly be used for VARCHAR2 and DATE, respectively.

Accessing object type information takes a few round-trips so the earlier methods are better for small numbers of values. If you use this solution in a more complex statement, check the optimizer plan is still OK. 